In [1]:
import tiktoken
import torch
torch.manual_seed(123)
tokenizer = tiktoken.get_encoding(encoding_name="gpt2")

# Input Text/Encoding

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [3]:
raw_text[:20]

'I HAD always thought'

In [4]:
tokenizer.decode([tokenizer.eot_token])

'<|endoftext|>'

In [5]:
text = raw_text[:20] + '<|endoftext|>'
encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
inputs = torch.tensor(encoded)
text, inputs

('I HAD always thought<|endoftext|>',
 tensor([   40,   367,  2885,  1464,  1807, 50256]))

# Token Embeddings

In [6]:
vocab_size = tokenizer.n_vocab
output_dim = 3
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [7]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings, token_embeddings.shape

(tensor([[ 1.3966e+00, -9.9491e-01, -1.5822e-03],
         [-1.1659e+00,  1.3834e-01, -9.8013e-01],
         [ 1.0122e+00, -6.5152e-01,  8.6052e-02],
         [ 2.5264e-01,  2.1043e+00, -6.4511e-01],
         [-3.1843e-01, -1.2264e-01, -3.9307e-02],
         [-3.1300e-01,  7.5582e-01, -1.2656e+00]], grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 3]))

# Position Embeddings

In [8]:
max_length = len(inputs)
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [9]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
pos_embeddings, pos_embeddings.shape

(tensor([[ 0.1583,  1.0919,  0.8055],
         [ 1.1262, -0.8857, -1.6315],
         [-0.2570,  0.7127, -0.3622],
         [ 0.9609, -1.3697,  0.1381],
         [-1.2365,  1.9319,  0.4730],
         [ 0.7365,  0.1316,  0.2379]], grad_fn=<EmbeddingBackward0>),
 torch.Size([6, 3]))

# Add them up!

In [10]:
input_embeddings = token_embeddings + pos_embeddings
input_embeddings, input_embeddings.shape

(tensor([[ 1.5549,  0.0969,  0.8039],
         [-0.0397, -0.7474, -2.6117],
         [ 0.7552,  0.0611, -0.2762],
         [ 1.2135,  0.7346, -0.5071],
         [-1.5549,  1.8092,  0.4337],
         [ 0.4235,  0.8874, -1.0277]], grad_fn=<AddBackward0>),
 torch.Size([6, 3]))

# Simple self-attention (no trainable weights)

In [11]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89], # Your
        [0.55, 0.87, 0.66], # journey
        [0.57, 0.85, 0.64], # starts
        [0.22, 0.58, 0.33], # with
        [0.77, 0.25, 0.10], # one
        [0.05, 0.80, 0.55], # step
    ]
)

In [12]:
query = inputs[1] # Second input token will be our query
query

tensor([0.5500, 0.8700, 0.6600])

In [13]:
inputs.shape

torch.Size([6, 3])

In [14]:
attn_scores_2 = torch.empty(inputs.shape[0])
attn_scores_2

tensor([2.3049e-26, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [15]:
query

tensor([0.5500, 0.8700, 0.6600])

In [30]:
for i, x_i in enumerate(inputs):
  attn_scores_2[i] = torch.dot(x_i, query)
attn_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [33]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=-1)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [42]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
query, query.shape, context_vec_2
for i, x_i in enumerate(inputs):
  context_vec_2 += attn_weights_2[i] * x_i
  print(f"{attn_weights_2[i] * x_i} = {attn_weights_2[i]:.2f} * {x_i}")
print(context_vec_2)

tensor([0.0596, 0.0208, 0.1233]) = 0.14 * tensor([0.4300, 0.1500, 0.8900])
tensor([0.1308, 0.2070, 0.1570]) = 0.24 * tensor([0.5500, 0.8700, 0.6600])
tensor([0.1330, 0.1983, 0.1493]) = 0.23 * tensor([0.5700, 0.8500, 0.6400])
tensor([0.0273, 0.0719, 0.0409]) = 0.12 * tensor([0.2200, 0.5800, 0.3300])
tensor([0.0833, 0.0270, 0.0108]) = 0.11 * tensor([0.7700, 0.2500, 0.1000])
tensor([0.0079, 0.1265, 0.0870]) = 0.16 * tensor([0.0500, 0.8000, 0.5500])
tensor([0.4419, 0.6515, 0.5683])


In [17]:
attn_scores = inputs @ inputs.T
mask = attn_scores == attn_scores.max(dim=0, keepdim=True).values
print(mask)

attn_scores = attn_scores * mask
print(attn_scores)

tensor([[ True, False, False, False, False, False],
        [False,  True,  True,  True, False,  True],
        [False, False, False, False,  True, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False],
        [False, False, False, False, False, False]])
tensor([[0.9995, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.4950, 1.4754, 0.8434, 0.0000, 1.0865],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7154, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])


In [26]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()
print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [20]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [23]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [25]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [43]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


In [45]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2
x_2, d_in, d_out

(tensor([0.5500, 0.8700, 0.6600]), 3, 2)

In [57]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [58]:
query_2 = x_2 @ W_query
key_2   = x_2 @ W_key
value_2 = x_2 @ W_value
query_2, key_2, value_2

(tensor([0.4306, 1.4551]), tensor([0.4433, 1.1419]), tensor([0.3951, 1.0037]))

In [59]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)
print(keys, query_2)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])
tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]]) tensor([0.4306, 1.4551])


In [60]:
keys_2 = keys[1]
attn_scores_22 = query_2.dot(keys_2)
attn_scores_22, keys_2

(tensor(1.8524), tensor([0.4433, 1.1419]))

In [76]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [77]:
print(query_2)
print(keys.T)

tensor([0.4306, 1.4551])
tensor([[0.3669, 0.4433, 0.4361, 0.2408, 0.1827, 0.3275],
        [0.7646, 1.1419, 1.1156, 0.6706, 0.3292, 0.9642]])


In [78]:
print(query_2, keys.T[:, 0], query_2 @ keys.T[:,0])

tensor([0.4306, 1.4551]) tensor([0.3669, 0.7646]) tensor(1.2705)


In [79]:
d_k = keys.shape[-1]
d_k

2

In [80]:
attn_weights_2  = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [81]:
values

tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])

In [82]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([0.3061, 0.8210])